### B. 线性规划与Word Mover's Distance

WMD在文本分析领域算作是一个比较经典的算法，它可以用来计算两个文本之间的相似度。 比如问答系统中，可以判断一个用户的query跟哪一个知识库里的问题最相近。而且，计算两个文本之间的相似度这个问题是NLP的核心，这也是为什么文本相似度计算这么重要的原因。 

背景： 在文本相似度匹配问题上如果使用tf-idf等模型，那这时候假如两个文本中没有出现共同的单词，则计算出来的相似度为0，但我们知道实际上很多时候单词可能不一样，但表示的内容确是类似的。 比如 ”People like this car“, "Those guys enjoy driving that", 虽然没有任何一样的单词，意思确是类似的。 这是WMD算法提出来的初衷。

WMD作为文本相似度计算的一种方法，最早由Matt J. Kusner, Yu Sun, Nicholas I. Kolkin, Kilian Q. Weinberger等人提出。但实际上它的想法极其简单，可以认为是Transportation Problem用在了词向量上， 其核心是线性规划。 对于Transportation问题在课上已经讲过，仍不清楚的朋友可以回顾一下课程的内容。 

在Section B里我们需要做两件事情： 1.  实现WMD算法来计算两个字符串之间的距离。  2. WMD的拓展方案

#### 1. WMD算法的实现
具体算法的实现是基于线性规划问题，细节请参考WMD的论文。 核心思想是把第一个句子转换成第二个句子过程中需要花费的最小cost。 

<img src="picture1.png" alt="drawing" width="600"/>  


线性规划问题即可以写成如下形式：

<img src="picture2.png" alt="drawing" width="500"/>



这里的参数是$T_{ij}$， 需要通过LP Solver去解决。$c(i,j)$指的是两个单词之间的距离, $c_{i,j}=||x_i-x_j||_2$。 参考： $||x||_2=\sqrt{x_1^2+...+x_d^2}$

为了实现WMD算法，首先需要词向量。 在这里，我们就不自己去训练了，直接使用已经训练好的词向量。 
请下载训练好的Glove向量：https://nlp.stanford.edu/projects/glove/  下载其中的 glove.6B.zip， 并使用d=100维的向量。 由于文件较大，需要一些时间来下载。 

请注意：提交作业时不要上传此文件， 但文件路径请使用我们给定的路径，不要改变。 

In [1]:
from scipy.spatial.distance import euclidean
from collections import Counter
from scipy.optimize import linprog
import numpy as np
import time
import re
from tqdm.notebook import tqdm

http://www.acme.byu.edu/wp-content/uploads/2014/09/Vol2Lab14.pdf

#### 先看看一个运输问题的例子
<img src="pic3.jpg" alt="drawing" width="600"/> 
<img src="pic4.jpg" alt="drawing" width="600"/>  

In [2]:
# 目标函数中各参数项的系数 Object = c^T x 
c = [6, 4, 6, 6, 5, 5]

# 等式约束条件：  Ax = b
A = [[1, 1, 1, 0, 0, 0], 
     [0, 0, 0, 1, 1, 1],
     [1, 0, 0, 1, 0, 0],
     [0, 1, 0, 0, 1, 0],
     [0, 0, 1, 0, 0, 1]
    ]
b = [200, 300, 150, 150, 200]

# X的定义域约束：
x_bounds = [(0, None)]*6

res = linprog(c, A_eq=A, b_eq=b, bounds=x_bounds, method='revised simplex')
res

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  app.launch_new_instance()


     con: array([0., 0., 0., 0., 0.])
     fun: 2500.0
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([ 50., 150.,   0., 100.,   0., 200.])

In [3]:
print("\n".join(["x{} = {}".format(ith, x) for ith, x in enumerate(res.x)]))
print("\nLowest cost: {}".format(res.fun))

x0 = 50.0
x1 = 150.0
x2 = 0.0
x3 = 100.0
x4 = 0.0
x5 = 200.0

Lowest cost: 2500.0


In [4]:
class Embedder:
    """
    用于维护Embeddings的类，提供Embeddings处理的一些接口。
    主要功能：
        1.读取并初始化Embedding
        2.给定单词获取Embedding
        3.通过Embedding计算词之间的距离
    """
    
    def __init__(self, glovefile, emb_dim=100, dist_fun=euclidean):
        self.w2vec = {}
        self.dist_fun = dist_fun
        self.emb_dim = 100
        self.load_embeddings(glovefile)
    
        
    def load_embeddings(self, file):
        """
        给定glove文件stream,读取embeddings
        
        :param file: 已经打开的embedding文件, 类型File
        """
        print("Loading Glove embeddings...")
        for line in tqdm(file):
            if not line: continue # 过滤空行
            items = line.strip().split(" ")
            assert(len(items) == 101)
            word = items[0]
            vector = np.array(items[1:], np.float)
            self.w2vec[word] = vector
        print("Finished loading embeddings.")
        
    
    def is_valid(self, word):
        """
        检查是否是词典词。
        
        :param word: 单词, 类型string
        :return: 判断结果, 类型Boolean
        """
        return word in self.w2vec
    
        
    def embed(self, word):
        """
        给定单词返回该单词的Embedding,若OOV，返回缺省词向量。
        
        :param word: 单词, 类型string
        :return: 词向量, 类型numpy.Array 
        """
        if self.is_valid(word):
            return self.w2vec[word]
        
        print("非词典词(OOV): {}".format(word))
        return np.zeros(self.emb_dim, dtype=np.float)
        
        
    def distance(self, w1, w2):
        """
        给定单词w1和w2,计算语义距离
        
        :param w1: 单词, 类型string
        :param w2: 单词, 类型string
        :return: 语义距离, 类型float
        """
        return self.dist_fun(self.embed(w1), self.embed(w2))

In [5]:
class PloblemFormalizer:
    """
    用于构建WMD优化问题的类。
    主要实现功能：
        1.构建词袋
        2.构建运输代价矩阵
        3.构建目标函数
        4.构建不等式约束
        5.构建等式约束
    """
    
    def __init__(self, embedder, stopwords=set([""])):
        """
        :param embedder: 词向量处理类的实例, 类型Embedder
        :param stopwords: 停用词表, 类型dict/set
        """
        self.embedder = embedder
        self.stopwords = stopwords
        self.split_pattern = "\.|,|;|。|,|!|\?|\(|\)|<|>|\*|&|%|\$|#|\s+" # 用于分词的pattern
        
        
    def nlp(self, sent):
        """
        对给定句子进行语言处理，设计分词，规范化，去除停用词等
        
        :param sent: 句子, 类型string
        :return: 分完词后的句子, 类型list[string]
        """
        words = []
        for token in re.split(self.split_pattern, sent):
            if token in self.stopwords or not token.strip():
                continue
                
            token = token.strip().lower() # 统一为小写
            
            # 如有需要可用nltk或spacy做lemmatization、stemming等规范化处理
            if not self.embedder.is_valid(token):
                print("Warning: 忽略非词典词（OOV）: {}".format(token))
                continue
            
            words.append(token)
        return words   
    
        
    def sent2bow(self, sent_words):
        """
        给定句子构建其词袋
        
        :param sent: nlp后的词, 类型list[string]
        :return: 词袋, 类型dict{string:float}
        """
        n_words = len(sent_words)
        counts = Counter(sent_words)
        return {w:count/n_words for w, count in counts.items()}
    
                
    def get_cost_mat(self, bow1, bow2):
        """
        给定两个词袋构建代价矩阵,元素为c(i,j)
        
        :param bow1: 词袋, 类型dict
        :param bow2: 词袋, 类型dict
        :return: 代价矩阵, 类型np.ndarray
        """
        n_sent1, n_sent2 = len(bow1), len(bow2)
        cost_mat = np.zeros((n_sent1, n_sent2), dtype=np.float)
        for ith, w1 in enumerate(bow1.keys()):
            for jth, w2 in enumerate(bow2.keys()):
                cost_mat[ith][jth] = self.embedder.distance(w1, w2)        
        return cost_mat
    
    
    def inequality_constraints(self, bow1, bow2):
        """
        给定词袋，构建WMD不等式约束 T_{i,j} >= 0
        
        :param bow1: 词袋, 类型dict
        :param bow2: 词袋, 类型dict
        :return: 不等式约束相关项, 类型list of tuple
        """
        bounds = [(0, None)] * (len(bow1) * len(bow2))
        return bounds

    
    def equality_constraints(self, bow1, bow2):
        """
        给定词袋，构建WMD等式约束：
            \sum_{j} T_{i,j} = d_i
            \sum_{i} T_{i,j} = d'_j
        
        :param bow1: 词袋, 类型dict
        :param bow2: 词袋, 类型dict
        :return: 等式约束相关项, 类型tuple(np.ndarray, np.array)
        """
        n_rows, n_cols = len(bow1), len(bow2)
        n_parameters = n_rows * n_cols
        n_words = n_rows + n_cols
        coef_mat = np.zeros((n_words, n_parameters), dtype=np.float)
        constants = np.zeros(n_words, dtype=np.float)
        
        # 构建句子1单词对应的等式约束
        for ith, word1 in enumerate(bow1.keys()):
            # 找到索引位置
            start_idx = ith * n_cols
            end_idx = start_idx + n_cols # excluded
            
            coef_mat[ith, start_idx:end_idx] = 1.0
            constants[ith] = bow1[word1]
            
        # 构建句子2单词对应的等式约束
        for jth, word2 in enumerate(bow2.keys()):
            # 找到索引位置
            indices = []
            for ith_row in range(n_rows):
                index = ith_row * n_cols + jth
                indices.append(index)
                                
            coef_mat[jth + n_rows, indices] = 1.0
            constants[jth + n_rows] = bow2[word2]
                    
        return coef_mat, constants 
        
    
    def get_objective(self, cost_mat):
        """
        给定代价矩阵，构建目标函数(参数系数): 
            \sum_{i,j} c(i,j)T_{i,j}
        
        :param cost_mat: 代价矩阵, 类型np.ndarray
        :return: 优化目标函数中的参数系数, 类型np.array
        """
        return cost_mat.flatten() # [[1,2],[3,4]] --> [1,2,3,4]

In [6]:
# 读取Glove文件。 注意： 不要试图修改文件以及路径
glovefile = open('glove.6B.100d.txt',"r",encoding="utf-8")  

# TODO: 编写WMD函数来计算两个句子之间的相似度

embedder = Embedder(glovefile)
prob_helper = PloblemFormalizer(embedder)

def WMD (sent1, sent2):
    """
    这是主要的函数模块。参数sent1是第一个句子， 参数sent2是第二个句子，可以认为没有经过分词。
    在英文里，用空格作为分词符号。
    
    在实现WMD算法的时候，需要用到LP Solver用来解决Transportation proboem. 原作业要求使用cvxopt工具，乔老师使用Scipy中的实现。
    大家可以尝试自己习惯的package
    
    需要做的事情为：
    
    1. 对句子做分词： 调用 .split() 函数即可
    2. 获取每个单词的词向量。这需要读取文件之后构建embedding matrix. 
    3. 构建lp问题，并用solver解决
    
    可以自行定义其他的函数，但务必不要改写WMD函数名。测试时保证WMD函数能够正确运行。
    """
    print("Sentence 1: {}".format(sent1), "Sentence 2: {}".format(sent2), sep="\n")
        
    sent1_bow = prob_helper.sent2bow(prob_helper.nlp(sent1)) # 句子->预处理->bow
    sent2_bow = prob_helper.sent2bow(prob_helper.nlp(sent2))
    
    if not len(sent1_bow) or not len(sent2_bow):
        print("至少有一个句子无任何有效单词，请重新输入")
        return None
        
    cost_mat = prob_helper.get_cost_mat(sent1_bow, sent2_bow)
    
    # 构建优化目标
    c = prob_helper.get_objective(cost_mat)
    
    # 构建不等式约束
    x_bounds = prob_helper.inequality_constraints(sent1_bow, sent2_bow)
        
    # 构建等式约束
    A, b = prob_helper.equality_constraints(sent1_bow, sent2_bow)
        
    # 将参数输入Solver求解
    res = linprog(c, A_eq=A, b_eq=b, bounds=x_bounds, method='revised simplex')
    
    wmd_dist = res.fun
    
    return wmd_dist
    
    

Loading Glove embeddings...



Finished loading embeddings.


In [7]:
def test():
    print("DISTANCE:\t", WMD("people like this car","those guys enjoy driving that"))
    print("*"*80)
    print("DISTANCE:\t", WMD("Obama speaks to the media in Illinois", "The President greets the press in Chicago"))
    print("*"*80)
    print("DISTANCE:\t", WMD("the capital of China is Beijing", "the capital of China is Beijing"))
    print("*"*80)
    print("DISTANCE:\t", WMD("i am studying the course of math ", "you are learning English lesson"))
    print("*"*80)
    print("DISTANCE:\t", WMD("the capital of China is Beijing", "Good morning, and in case I don't see you, good afternoon, good evening, and good night!"))
test()

Sentence 1: people like this car
Sentence 2: those guys enjoy driving that
DISTANCE:	 4.271249445939769
********************************************************************************
Sentence 1: Obama speaks to the media in Illinois
Sentence 2: The President greets the press in Chicago
DISTANCE:	 3.489272451179516
********************************************************************************
Sentence 1: the capital of China is Beijing
Sentence 2: the capital of China is Beijing
DISTANCE:	 0.0
********************************************************************************
Sentence 1: i am studying the course of math 
Sentence 2: you are learning English lesson
DISTANCE:	 5.016964993274135
********************************************************************************
Sentence 1: the capital of China is Beijing
Sentence 2: Good morning, and in case I don't see you, good afternoon, good evening, and good night!
DISTANCE:	 5.642871935596172


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.


#### 2. WMD算法的拓展


##### 2.1 从欧式距离到Mahalanobis距离
WMD算法本身不需要任何的标注好的数据，所以它属于无监督学习。 而且在上述的WMD算法里使用的是欧式距离，$c(i,j)=||x_i-x_j||_2$， 那这种距离有什么缺点呢？ 其中一个缺点是欧式距离的计算把一个空间里的每一个维度都看成了同样的权重，也就是每一个维度的重要性都是一致的，而且不同维度之间的相关性也没有考虑进来。如果想把这些信息考虑进来，我们则可以使用一个改进版的距离计算叫做Mahalanobis Distance， 距离计算变成 $c(i,j)=(x_i-x_j)^{\top}M(x_i-x_j)$。

这如何去理解呢？ Mahalanobis distance可以理解成： 首先我们对原始空间里的样本做了一层线性的转换， 然后在转换后的空间里计算欧式距离。 我们把这个过程写一下： 原始空间里的点为 $x_i$, 然后我们定义一个转换矩阵 $L$, 这时候就可以得到 $||Lx_i - Lx_j||_2^2=||L(x_i-x_j)||_2^2=(L(x_i-x_j))^{\top}L(x_i-x_j)=(x_i-x_j)^{\top}L^{\top}L(x_i-x_j)=(x_i-x_j)^{\top}M(x_i-x_j)$, 相当于把$L^{\top}L$看做是矩阵$M$。这时候很容易看出来矩阵$M$是PSD(positive semidefinite). 

假设我们定义了这种距离，这里的M如何选择呢？ 当然，这是需要学出来的！ 那为了学出M， 必须要有标注好的训练数据，也就需要监督学习场景！  

##### 2.2 从无监督学习到监督学习

假如拥有数据集$D={(s_1, y_1),...,(s_n, y_n)}$， 这里每一个$s_i$代表的是一个句子， $y_i$代表的是对应每一个句子的标签（label)。 我们希望使用这个数据来学出M的值。那如何学习呢？ 在这个问题上能使用的方法其实比较多，但在这里， 我们采用一个margin-based方法，这一点在SVM里面其实接触过。

具体一点，假如我们手里有三个句子，$s_u, s_v, s_w$, 其中$s_u$和$s_v$是属于同一个类别，$s_w$是属于另一个类别，那这时候从KNN的角度来讲，我们希望$s_u, s_v$的距离要小于 $s_u, s_w$之间的距离。 用数学来表示： $d(s_u, s_v) < d(s_u, s_w)$, $~~d(.,.)$表示两个文本之间的距离。 其实我们希望它们之间的距离越大越好，也就是所谓的完全区间越宽越好。 但实际上，这个距离太大也没有什么意义，所以我们就干脆指定一个参数 $\eta$来表示margin, 也就是只要它俩之间的距离大于这个margin就可以。如果小于margin就给他们一些惩罚（penalty)，这一点跟SVM极其相似（slack variable）。所以从这个角度SVM也叫做margin-based classifier. 

把上述的表示成数学的话： $d(s_u, s_v) + \eta < d(s_u, s_k)$, 但如果这个式子不成立的话就可以认为产生了penalty。 所以这部分就可以表示成大家熟悉的hinge loss:    $max (0,  d(s_u, s_v) + \eta - d(s_u, s_k))$。 另外，我们同时也希望如果两个样本属于同一个类别， 那它俩的距离也比较相近。所以目标函数可以分为两个部分： 1. 同类型的样本距离尽量要近   2. 不同类型的样本距离尽量远一些。 

当我们把所有的样本以及他们之间的大小关系考虑进来之后就可以得到最终的目标函数。 

\begin{equation}
L = \lambda \sum_{u=1}^{n}\sum_{v\in pos(u)}d(s_u, s_v) + (1-\lambda)\sum_{u=1}^{n}\sum_{v\in pos(u)}^{}\sum_{w\in neg(u)}^{} max (0,  d(s_u, s_v) + \eta - d(s_u, s_w))
\end{equation}

这里几个notation:  pos(u)代表的是跟样本u属于同一个类别的样本， neg(u)指的是跟样本u属于不同类别的样本。 注意：类别的个数可以大于2, 就是多分类问题。 你也可以参考： http://jmlr.org/papers/volume10/weinberger09a/weinberger09a.pdf

在这个式子里，第一部分代表的是让同一类型的样本的距离变小， 第二部分代表的是不同类型的样本之间要扩大距离。 

- #### Q1: 这里$\lambda$起到什么作用？

$\lambda$,是一个超参数，目的是调节两个目标函数，决定了那个优化目标的权重更高

- #### Q2: 在目标函数里有$\eta$值，这个值怎么理解？ 如何去设定这个值呢？


$\eta$就是类间距离和类别距离之差,可以表示为不同类别之间可以设置的最大距离。调参

这里的$d_{u,v}$指的是$s_u$和$s_v$之间的距离， 而且这个距离被定义为：

\begin{equation} d_{u, v}=min_{T\geq 0}\sum_{i,j}^{}T_{ij}c(i,j)^u~~~~  s.t. \sum_{j=1}^{}T_{ij}=d_i^u, ~~\sum_{i=1}^{}T_{ij}=d_j'^v\end{equation}

这里  $c(i,j)=(x_i-x_j)^{\top}M(x_i-x_j)$。 所以是不是可以察觉到这个问题目标函数里既包含了参数$M$也包含了线性规划问题。

- #### Q3: 请试着去理解上述所有的过程，并回答： 优化问题如何解决呢？ 请给出解题的思路 （文字适当配合推导过程）。 

首先我们设定一个初始的M矩阵，然后一旦M确定下来，计算du,vdu,v的方法就回到了我们之前解决过的wmd问题的框架里面。从而我们继续利用LP办法求出按照当下M矩阵算出来的最小距离du,vdu,v，同样我们也可以用这个M计算du,wdu,w。至此，我们已经得到了两个不同的distance，我们就可以将其带入到目标函数里面去，假设我们的超参数已经设定好，我们便可以求出目标函数。然后可以利用梯度下降法，将得到的梯度传递到M里面，并且根据此来优化我们M的初始值。

对于上述问题，其实我们也可以采用不一样的损失函数来求解M。 一个常用的损失函数叫作 “kNN-LOO error”， 相当于把KNN的准确率转换成了smooth differential loss function. 感兴趣的朋友可以参考： https://papers.nips.cc/paper/6139-supervised-word-movers-distance.pdf



以上是优化部分的一个简短的作业，通过这些练习会对优化理论有更清晰的认知。  Good luck for everyone！ 